In [1]:
import requests
import csv
import statistics

res = requests.get("https://api.blockchain.info/charts/hash-rate?cors=true&format=csv&lang=de").text
lines = list(csv.reader(res.splitlines()))

# Average over the last 60 days and convert from THash/s to Hash/s
hashrate_60d_avg = statistics.mean(float(rate_s)*10e12 for _, rate_s in lines[-60:]) # Hash/s


https://en.bitcoin.it/wiki/Mining_hardware_comparison#cite_note-AntMinerS9-9
https://www.bitmaintech.com/productDetail.htm?pid=0002016052907243375530DcJIoK0654

In [2]:

# Very optimistic lower bound, ignoring older hardware, cooling etc. pp.
power_consumption = 0.098e-9 # J/Hash

# 1J = 1Ws → Hash/s * J/Hash = J/s = Ws/s = W
network_power_consumption = hashrate_60d_avg * power_consumption # W

print('Total network power consumption (estimated lower bound):', network_power_consumption*1e-6, 'MW')

Total network power consumption (estimated lower bound): 1756.5274955466207 MW


In [5]:
import time

ms_per_d = 24*3600*1000
for ts in range(int(time.time()*1000 - 70*ms_per_d), ms_per_d):
    res = "https://blockchain.info/blocks/{}?format=json".format(ts).json
    time.sleep(1)

In [ ]:
import time

def fetch_blocks(days=70): # approx. 10k blocks in 70d
    ms_per_d = 1000 * 3600 * 24
    for i in range(int(time.time()*1000) - days*ms_per_d, int(time.time()*1000), ms_per_d):
        print(i)
        res = requests.get("https://blockchain.info/blocks/{}?format=json".format(i)).json()
        for e in res['blocks']:
            yield e['hash']
        time.sleep(0.6)
blocks = list(fetch_blocks())

1472142017412
1472228417412
1472314817412
1472401217412
1472487617412
1472574017412
1472660417412
1472746817412
1472833217412
1472919617412
1473006017412
1473092417412
1473178817412
1473265217412
1473351617412
1473438017412
1473524417412
1473610817412
1473697217412
1473783617412
1473870017412
1473956417412
1474042817412
1474129217412
1474215617412
1474302017412
1474388417412
1474474817412
1474561217412
1474647617412
1474734017412
1474820417412
1474906817412
1474993217412
1475079617412
1475166017412
1475252417412
1475338817412
1475425217412
1475511617412
1475598017412
1475684417412
1475770817412
1475857217412
1475943617412
1476030017412
1476116417412
1476202817412
1476289217412
1476375617412
1476462017412
1476548417412
1476634817412
1476721217412
1476807617412
1476894017412
1476980417412

In [11]:
def fetch_block_ips(blocks):
    n_err = 0
    for block in blocks:
        try:
            res = requests.get('https://blockchain.info/inv/{}?format=json'.format(block)).json()
            yield res['initial_ip']
        except:
            n_err += 1
        time.sleep(0.6)
    print(n_err, 'blocks w/o inventory info')

block_ip_fetch = []
ivl = 100
for i in range(0, 10000, ivl):
    print('Fetching', i)
    block_ip_fetch.append(list(fetch_block_ips(blocks[i:i+ivl])))

Fetching 0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)